# Lecture 5-2

## Pandas: Summaries with Pivot Tables and Group by

## Week 5 Wednesday

## Miles Chen, PhD

Adapted from Python Data Science Handbook by Jake VanderPlas and Python for Data Analysis by Wes McKinney

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.__version__

'2.2.2'

# Some important data transformation tools

## Multi Index, Hierarchical Indexing

In [3]:
np.random.seed(1)
# manual multi-index creation:
# provide the index argument with a list of lists
data = pd.Series(np.array([4, 5, 6, 8, 10, 12, 12, 15, 18]), 
                 index = [['a','a','a','b','b','b','c','c','c'],
                          [ 4 , 5 , 6 , 4 , 5 , 6 , 4 , 5 , 6 ]])

In [4]:
data

a  4     4
   5     5
   6     6
b  4     8
   5    10
   6    12
c  4    12
   5    15
   6    18
dtype: int64

In [5]:
data.index

MultiIndex([('a', 4),
            ('a', 5),
            ('a', 6),
            ('b', 4),
            ('b', 5),
            ('b', 6),
            ('c', 4),
            ('c', 5),
            ('c', 6)],
           )

In [6]:
# select via the outer index
data.loc['b']

4     8
5    10
6    12
dtype: int64

In [7]:
# select via the inner index
data.loc[:,5] 

a     5
b    10
c    15
dtype: int64

In [8]:
type(data.loc[:,5])

pandas.core.series.Series

In [9]:
data.loc[:,5].index

Index(['a', 'b', 'c'], dtype='object')

In [10]:
# the unstack function returns a new DataFrame where the values have been unstacked
# similar to tidyr's spread()/pivot_wider function in R
data.unstack()

,4,5,6
a,4,5,6
b,8,10,12
c,12,15,18


In [11]:
# after unstacking, the index is no longer a multi index
data.unstack().index

Index(['a', 'b', 'c'], dtype='object')

In [12]:
data.unstack().shape

(3, 3)

In [13]:
# the inverse operation of unstack() is stack()
# applying both of these functions will return the same series
data.unstack().stack()

a  4     4
   5     5
   6     6
b  4     8
   5    10
   6    12
c  4    12
   5    15
   6    18
dtype: int64

In [18]:
# you can swap the levels of the multi index using swaplevel
data.swaplevel()

4  a     4
5  a     5
6  a     6
4  b     8
5  b    10
6  b    12
4  c    12
5  c    15
6  c    18
dtype: int64

In [15]:
# the .loc accessors work as expected
data.swaplevel().loc[:,'a']

4    4
5    5
6    6
dtype: int64

In [19]:
# swaplevel will keep the original order
# you may want to sort based on the new swapped index levels
# you must save the output as data remains unchanged
data.swaplevel().sort_index()

4  a     4
   b     8
   c    12
5  a     5
   b    10
   c    15
6  a     6
   b    12
   c    18
dtype: int64

In [20]:
print(data)

a  4     4
   5     5
   6     6
b  4     8
   5    10
   6    12
c  4    12
   5    15
   6    18
dtype: int64


In [21]:
data.swaplevel().unstack()

,a,b,c
4,4,8,12
5,5,10,15
6,6,12,18


In [22]:
# compare to:
data.unstack()

,4,5,6
a,4,5,6
b,8,10,12
c,12,15,18


In [24]:
# summing and other aggregate functions can be performed on an index-based level
# calling sum() on a series, will sum the whole series
data.sum()

np.int64(90)

In [25]:
# you can call groupby on level 0 (the first level of the index) and then sum
# we get sums for each value in the first level of the index
# we will cover groupby in more detail later
data.groupby(level = 0).sum()

a    15
b    30
c    45
dtype: int64

In [26]:
data.groupby(level = 1).sum()

4    24
5    30
6    36
dtype: int64

# Reshaping and Pivoting Data

In [27]:
data = pd.DataFrame(np.arange(1,7).reshape((2, 3)),
                    index  = pd.Index(['alpha', 'beta'], name='letter'),
                    columns= pd.Index(['one', 'two', 'three'], name = 'number'))
data

number,one,two,three
letter,,,
alpha,1,2,3
beta,4,5,6


In [28]:
data.stack()  # creates a multi-index

letter  number
alpha   one       1
        two       2
        three     3
beta    one       4
        two       5
        three     6
dtype: int64

In [29]:
data.stack().unstack()  # unstack undoes the creation of the stacks

number,one,two,three
letter,,,
alpha,1,2,3
beta,4,5,6


In [32]:
data.stack().unstack(0) # you can specify how the unstacking should be done
# here we specify that we should unstack the first level of the multi-index

letter,alpha,beta
number,,
one,1,4
two,2,5
three,3,6


In [33]:
data.stack().unstack('letter')
# you can specify the unstacking by the index level name

letter,alpha,beta
number,,
one,1,4
two,2,5
three,3,6


In [34]:
data.stack().unstack('number')

number,one,two,three
letter,,,
alpha,1,2,3
beta,4,5,6


### Unstacking can introduce missing values

In [35]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])  
# using the argument keys when concat series will produce a multi-index
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [36]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [37]:
data2.unstack().stack() # stack() will filter out missing values

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

# Small example data wrangling

In [38]:
data = pd.read_csv('macrodata.csv')

https://www.statsmodels.org/dev/datasets/generated/macrodata.html

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      203 non-null    float64
 1   quarter   203 non-null    float64
 2   realgdp   203 non-null    float64
 3   realcons  203 non-null    float64
 4   realinv   203 non-null    float64
 5   realgovt  203 non-null    float64
 6   realdpi   203 non-null    float64
 7   cpi       203 non-null    float64
 8   m1        203 non-null    float64
 9   tbilrate  203 non-null    float64
 10  unemp     203 non-null    float64
 11  pop       203 non-null    float64
 12  infl      203 non-null    float64
 13  realint   203 non-null    float64
dtypes: float64(14)
memory usage: 22.3 KB


In [40]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


https://pandas.pydata.org/pandas-docs/stable/generated/pandas.PeriodIndex.html

In [41]:
# We can create a time based index of periods consisting of the year and quarter
periods = pd.PeriodIndex.from_fields(year = data.year, quarter = data.quarter)

In [42]:
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2', '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3', '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', length=203)

In [43]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name = 'item')
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [44]:
data = data.reindex(columns = columns) # forces columns to conform to the column index we specified

In [45]:
data.head(10)

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2
5,2834.390,0.14,5.2
6,2839.022,2.70,5.6
7,2802.616,1.21,6.3
8,2819.264,-0.40,6.8
9,2872.005,1.47,7.0


In [46]:
periods.to_timestamp('D','start')  # changes 1959Q1 to a date: the start date of Q1 of 1959: 1959-01-01

DatetimeIndex(['1959-01-01', '1959-04-01', '1959-07-01', '1959-10-01', '1960-01-01', '1960-04-01', '1960-07-01', '1960-10-01',
               '1961-01-01', '1961-04-01',
               ...
               '2007-04-01', '2007-07-01', '2007-10-01', '2008-01-01', '2008-04-01', '2008-07-01', '2008-10-01', '2009-01-01',
               '2009-04-01', '2009-07-01'],
              dtype='datetime64[ns]', length=203, freq='QS-OCT')

In [47]:
# the current index is just integers, and we want to replace it
data.index

RangeIndex(start=0, stop=203, step=1)

In [48]:
# specify a new index directly
data.index = periods.to_timestamp('D','start')

In [49]:
data.head()

item,realgdp,infl,unemp
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [50]:
data.stack().head(10)  # stack creates a series

            item   
1959-01-01  realgdp    2710.349
            infl          0.000
            unemp         5.800
1959-04-01  realgdp    2778.801
            infl          2.340
            unemp         5.100
1959-07-01  realgdp    2775.488
            infl          2.740
            unemp         5.300
1959-10-01  realgdp    2785.204
dtype: float64

In [51]:
data.stack().reset_index().head()
# calling reset index turns the current index into a new column and creates a new index

,level_0,item,0
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340


In [52]:
data.stack().reset_index().index

RangeIndex(start=0, stop=609, step=1)

In [53]:
# we'll stack the data into long form 
# and create an object called ldata, the long-version of data
ldata = data.stack().reset_index().rename(columns = {'level_0':'date', 0: 'value'})  
# rename changes the column title '0' to 'value'
ldata.head(10)

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


In [54]:
# the shape of ldata is 609 by 3
# there are 609 rows - each date has three rows
# the three columns are date, item name, value
ldata.shape

(609, 3)

In [55]:
# unstack doesn't work, because the stacking and unstacking is powered by multi-index
# instead, all the dates, item names, and values get 'flowed' into one column
# Notice the length is now 1827
ldata.unstack()

date   0      1959-01-01 00:00:00
       1      1959-01-01 00:00:00
       2      1959-01-01 00:00:00
       3      1959-04-01 00:00:00
       4      1959-04-01 00:00:00
                     ...         
value  604                   3.37
       605                    9.2
       606              12990.341
       607                   3.56
       608                    9.6
Length: 1827, dtype: object

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html

#### updated: you must specify index, columns and values in pivot

In [56]:
# if the data is in 'long' form, you can change it to 'wide' form with pivot
ldata.pivot(index = 'date',columns = 'item',values = 'value').head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [57]:
# if the data is in 'long' form, you can change it to 'wide' form with pivot
# in this example, we specify the index to be 'item', of which there are three
# the date values become the columns, of which there are 203
ldata.pivot(index = 'item',columns = 'date',values = 'value').head()

date,1959-01-01,1959-04-01,1959-07-01,1959-10-01,1960-01-01,1960-04-01,1960-07-01,1960-10-01,1961-01-01,1961-04-01,1961-07-01,1961-10-01,1962-01-01,1962-04-01,1962-07-01,1962-10-01,1963-01-01,1963-04-01,1963-07-01,1963-10-01,1964-01-01,1964-04-01,1964-07-01,1964-10-01,1965-01-01,1965-04-01,1965-07-01,1965-10-01,1966-01-01,1966-04-01,1966-07-01,1966-10-01,1967-01-01,1967-04-01,1967-07-01,1967-10-01,1968-01-01,1968-04-01,1968-07-01,1968-10-01,1969-01-01,1969-04-01,1969-07-01,1969-10-01,1970-01-01,1970-04-01,1970-07-01,1970-10-01,1971-01-01,1971-04-01,1971-07-01,1971-10-01,1972-01-01,1972-04-01,1972-07-01,1972-10-01,1973-01-01,1973-04-01,1973-07-01,1973-10-01,1974-01-01,1974-04-01,1974-07-01,1974-10-01,1975-01-01,1975-04-01,...,1993-04-01,1993-07-01,1993-10-01,1994-01-01,1994-04-01,1994-07-01,1994-10-01,1995-01-01,1995-04-01,1995-07-01,1995-10-01,1996-01-01,1996-04-01,1996-07-01,1996-10-01,1997-01-01,1997-04-01,1997-07-01,1997-10-01,1998-01-01,1998-04-01,1998-07-01,1998-10-01,1999-01-01,1999-04-01,1999-07-01,1999-10-01,2000-01-01,2000-04-01,2000-07-01,2000-10-01,2001-01-01,2001-04-01,2001-07-01,2001-10-01,2002-01-01,2002-04-01,2002-07-01,2002-10-01,2003-01-01,2003-04-01,2003-07-01,2003-10-01,2004-01-01,2004-04-01,2004-07-01,2004-10-01,2005-01-01,2005-04-01,2005-07-01,2005-10-01,2006-01-01,2006-04-01,2006-07-01,2006-10-01,2007-01-01,2007-04-01,2007-07-01,2007-10-01,2008-01-01,2008-04-01,2008-07-01,2008-10-01,2009-01-01,2009-04-01,2009-07-01
item,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
infl,0.000,2.340,2.740,0.270,2.310,0.14,2.700,1.210,-0.400,1.470,0.800,0.80,2.260,0.130,2.110,0.790,0.530,2.750,0.780,2.460,0.130,0.900,1.290,2.050,1.280,2.54,0.890,2.900,4.990,2.100,4.900,0.61,2.42,3.610,3.580,4.72,3.500,5.770,4.560,4.510,6.67,5.47,5.400,6.380,6.280,4.130,5.110,5.040,2.000,4.960,2.940,2.920,2.900,2.880,3.810,4.710,9.260,4.550,12.470,10.39,10.960,9.860,13.56,10.07,5.320,7.480,...,1.940,3.030,1.920,2.450,3.250,2.690,2.93,3.440,2.100,2.350,3.110,3.600,2.300,3.050,3.020,1.250,1.250,2.730,1.240,0.490,2.460,1.710,1.950,2.900,1.920,3.350,2.850,3.760,4.190,2.770,3.890,1.820,2.260,0.450,0.230,3.590,1.56,2.66,3.080,1.310,1.090,2.600,3.020,2.350,3.610,3.580,2.090,4.150,1.850,9.140,0.400,2.600,3.970,-1.580,3.300,4.580,2.750,3.450,6.380,2.820,8.530,-3.16,-8.79,0.94,3.370,3.560
realgdp,2710.349,2778.801,2775.488,2785.204,2847.699,2834.39,2839.022,2802.616,2819.264,2872.005,2918.419,2977.83,3031.241,3064.709,3093.047,3100.563,3141.087,3180.447,3240.332,3264.967,3338.246,3376.587,3422.469,3431.957,3516.251,3563.96,3636.285,3724.014,3815.423,3828.124,3853.301,3884.52,3918.74,3919.556,3950.826,3980.97,4063.013,4131.998,4160.267,4178.293,4244.10,4256.46,4283.378,4263.261,4256.573,4264.289,4302.259,4256.637,4374.016,4398.829,4433.943,4446.264,4525.769,4633.101,4677.503,4754.546,4876.166,4932.571,4906.252,4953.05,4909.617,4922.188,4873.52,4854.34,4795.295,4831.942,...,8486.435,8531.108,8643.769,8727.919,8847.303,8904.289,9003.18,9025.267,9044.668,9120.684,9184.275,9247.188,9407.052,9488.879,9592.458,9666.235,9809.551,9932.672,10008.874,10103.425,10194.277,10328.787,10507.575,10601.179,10684.049,10819.914,11014.254,11043.044,11258.454,11267.867,11334.544,11297.171,11371.251,11340.075,11380.128,11477.868,11538.77,11596.43,11598.824,11645.819,11738.706,11935.461,12042.817,12127.623,12213.818,12303.533,12410.282,12534.113,12587.535,12683.153,12748.699,12915.938,12962.462,12965.916,13060.679,13099.901,13203.977,13321.109,13391.249,13366.865,13415.266,13324.60,13141.92,12925.41,12901.504,12990.341
unemp,5.800,5.100,5.300,5.600,5.200,5.20,5.600,6.300,6.800,7.000,6.800,6.20,5.600,5.500,5.600,5.500,5.800,5.700,5.500,5.600,5.500,5.200,5.000,5.000,4.900,4.70,4.400,4.100,3.900,3.800,3.800,3.70,3.80,3.800,3.800,3.90,3.700,3.500,3.500,3.400,3.40,3.40,3.600,3.600,4.200,4.800,5.200,5.800,5.900,5.900,6.000,6.000,5.800,5.700,5.600,5.300,5.000,4.900,4.800,4.80,5.100,5.200,5.60,6.6

In [58]:
# all of these pivot operations return new objects and leave the original data unchanged
data.head()

item,realgdp,infl,unemp
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


# Group By


In [59]:
pd.__version__

'2.2.2'

In [66]:
np.random.seed(1)
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randint(20, size = 5),
                   'data2' : np.random.randint(20, size = 5)})
df

,key1,key2,data1,data2
0,a,one,5,11
1,a,two,11,5
2,b,one,12,15
3,b,two,8,0
4,a,one,9,16


In [67]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [68]:
grouped.mean()

key1
a     8.333333
b    10.000000
Name: data1, dtype: float64

In [69]:
df

,key1,key2,data1,data2
0,a,one,5,11
1,a,two,11,5
2,b,one,12,15
3,b,two,8,0
4,a,one,9,16


#### updated: if there is a mix of numeric and categorical data, specify `numeric_only`

In [70]:
df.groupby(by = 'key1').mean(numeric_only = True)
# if you don't specify the column, it'll apply the function to the entire dataframe

,data1,data2
key1,,
a,8.333333,10.666667
b,10.000000,7.500000


In [71]:
df

,key1,key2,data1,data2
0,a,one,5,11
1,a,two,11,5
2,b,one,12,15
3,b,two,8,0
4,a,one,9,16


In [73]:
means = df.groupby([df['key1'], df['key2']]).mean()
means
# means has a multi-index

data1  data2
key1 key2              
a    one     7.0   13.5
     two    11.0    5.0
b    one    12.0   15.0
     two     8.0    0.0

In [74]:
# with the multi-index, you can unstack
means.unstack()

data1       data2     
key2   one   two   one  two
key1                       
a      7.0  11.0  13.5  5.0
b     12.0   8.0  15.0  0.0

In [75]:
df

,key1,key2,data1,data2
0,a,one,5,11
1,a,two,11,5
2,b,one,12,15
3,b,two,8,0
4,a,one,9,16


In [76]:
# you can perform group by on Series that are not in the dataframe, but are of the correct length
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    11.0
            2006    12.0
Ohio        2005     6.5
            2006     9.0
Name: data1, dtype: float64

In [77]:
df

,key1,key2,data1,data2
0,a,one,5,11
1,a,two,11,5
2,b,one,12,15
3,b,two,8,0
4,a,one,9,16


In [78]:
df.groupby(['key1', 'key2']).size() # you don't always have to use mean, you can use other functions as well

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating over groups

In [79]:
df

,key1,key2,data1,data2
0,a,one,5,11
1,a,two,11,5
2,b,one,12,15
3,b,two,8,0
4,a,one,9,16


In [80]:
# the groupby creates a series of tuples that can be unpacked into name and group
for name, group in df.groupby('key1'):
    print("name:", name)
    print('------')
    print("group:\n", group)
    print('------')
    print("data1 mean:", group.data1.mean())
    print("data2 mean:", group.data2.mean())
    print('**************************')

name: a
------
group:
   key1 key2  data1  data2
0    a  one      5     11
1    a  two     11      5
4    a  one      9     16
------
data1 mean: 8.333333333333334
data2 mean: 10.666666666666666
**************************
name: b
------
group:
   key1 key2  data1  data2
2    b  one     12     15
3    b  two      8      0
------
data1 mean: 10.0
data2 mean: 7.5
**************************


In [81]:
for name, group in df.groupby('key2'):
    print("name:", name)
    print('------')
    print("group:\n", group)
    print('------')
    print("data1 mean:", group.data1.mean())
    print("data2 mean:", group.data2.mean())
    print('**************************')


name: one
------
group:
   key1 key2  data1  data2
0    a  one      5     11
2    b  one     12     15
4    a  one      9     16
------
data1 mean: 8.666666666666666
data2 mean: 14.0
**************************
name: two
------
group:
   key1 key2  data1  data2
1    a  two     11      5
3    b  two      8      0
------
data1 mean: 9.5
data2 mean: 2.5
**************************
